In [1]:
import os
import cv2
import mediapipe as mp
import csv

2024-05-01 20:51:42.836288: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 20:51:43.921214: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
keypoints = {
    23: 'left_hip',
    24: 'right_hip',
    25: 'left_knee',
    26: 'right_knee',
    27: 'left_ankle',
    28: 'right_ankle',
    29: 'left_heel',
    30: 'right_heel'
}

# mediapipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2, enable_segmentation=False)
mp_drawing = mp.solutions.drawing_utils  

I0000 00:00:1714564305.157094  116147 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1714564305.207494  116263 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.67), renderer: NVIDIA GeForce RTX 2080 Ti/PCIe/SSE2


In [3]:
def process_video(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Unable to open video: {video_path}")
        return

    video_name = os.path.splitext(os.path.basename(video_path))[0]
    csv_path = os.path.join(output_dir, f"{video_name}_keypoints.csv")

    # Prepare CSV headers
    headers = ['video_id', 'frame_id']
    for idx in sorted(keypoints.keys()):
        headers.extend([
            f"{keypoints[idx]}_x",
            f"{keypoints[idx]}_y",
            f"{keypoints[idx]}_z",
            f"{keypoints[idx]}_visibility"
        ])

    with open(csv_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)

        frame_id = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                row = [video_name, frame_id]
                landmark_dict = {lmk: (0, 0, 0, 0) for lmk in keypoints}  # Default values
                for landmark_id, landmark in enumerate(results.pose_landmarks.landmark):
                    if landmark_id in keypoints:
                        landmark_dict[landmark_id] = (landmark.x, landmark.y, landmark.z, landmark.visibility)

                for idx in sorted(keypoints.keys()):
                    row.extend(landmark_dict[idx])

                writer.writerow(row)

            cv2.imshow('Pose Estimation', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            frame_id += 1

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def process_all_videos(source_dir, output_dir):
    for file in os.listdir(source_dir):
        if file.endswith('.mp4'):
            video_path = os.path.join(source_dir, file)
            process_video(video_path, output_dir)
            print(f"Processed {file}")

In [5]:
source = '/home/rafi/Desktop/GradResearch/ZeroSkate/NewTricks/Kickflip' 
new = '/home/rafi/Desktop/GradResearch/ZeroSkate/NewTricks/KickflipCSV' 
process_all_videos(source, new)

Processed Kickflip16.mp4
Processed Kickflip57.mp4
Processed Kickflip6.mp4
Processed Kickflip28.mp4
Processed Kickflip66.mp4
Processed Kickflip71.mp4
Processed Kickflip34.mp4
Processed Kickflip58.mp4
Processed Kickflip102.mp4
Processed Kickflip77.mp4
Processed Kickflip103.mp4
Processed Kickflip61.mp4
Processed Kickflip72.mp4
Processed Kickflip15.mp4
Processed Kickflip49.mp4
Processed Kickflip87.mp4
Processed Kickflip23.mp4
Processed Kickflip60.mp4
Processed Kickflip100.mp4
Processed Kickflip11.mp4
Processed Kickflip84.mp4
Processed Kickflip31.mp4
Processed Kickflip3.mp4
Processed Kickflip90.mp4
Processed Kickflip65.mp4
Processed Kickflip42.mp4
Processed Kickflip24.mp4
Processed Kickflip94.mp4
Processed Kickflip36.mp4
Processed Kickflip104.mp4
Processed Kickflip33.mp4
Processed Kickflip92.mp4
Processed Kickflip107.mp4
Processed Kickflip99.mp4
Processed Kickflip5.mp4
Processed Kickflip89.mp4
Processed Kickflip21.mp4
Processed Kickflip52.mp4
Processed Kickflip47.mp4
Processed Kickflip54.mp